In [2]:
import os.path as osp

from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import models


https://choice37.tistory.com/27

In [6]:
print(torch.cuda.device_count())
torch.cuda.get_device_name(0)
torch.cuda.is_available()

0


AssertionError: Torch not compiled with CUDA enabled

### Dataset, DataLoader

In [2]:
from utils.dataloader_image_classification import ImageTransform,make_datapath_list,HymenopteraDataset

train_list=make_datapath_list()
val_list=make_datapath_list(phase="val")

size,mean,std=2242,(0.485,0.456,0.406),(0.229,0.224,0.225)
train_dataset=HymenopteraDataset(file_list=train_list,transform=ImageTransform(size,mean,std))
val_dataset=HymenopteraDataset(file_list=val_list,transform=ImageTransform(size,mean,std),phase="val")

batch_size=32
train_dataloader=data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader=data.DataLoader(val_dataset,batch_size=batch_size)

dataloaders_dict={"train": train_dataloader,"val":val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


### Model Structure

In [3]:
#create model instance
net=models.vgg16(pretrained=True)

#change output layer for fine tuning
net.classifier[6]=nn.Linear(in_features=4096,out_features=2)

#set train mode
net.train()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Loss function 

In [4]:
criterion=nn.CrossEntropyLoss()

### Optimizer
- 모든 layer의 parameter 학습 가능하도록 작성

In [5]:
#학습할 파라미터 (with fine tuning)
params_to_update=[[] for i in range(3)]

#학습할 층의 파라미터이름 지정
update_param_name=[["features"],["classifier.0.bias","classifier.3.weight","classifier.3.bias"],["classifier.6.weight","classifier.6.bias"]]

#각 리스트에 파라미터 저장
for name,param in net.named_parameters():
    for i in range(3):
        if update_param_name[i][0] in name or name in update_param_name[i]:
            param.requires_grad=True
            params_to_update[i].append(param)
            print(f"params to update[{i+1}]에 {name} 저장")

params to update[1]에 features.0.weight 저장
params to update[1]에 features.0.bias 저장
params to update[1]에 features.2.weight 저장
params to update[1]에 features.2.bias 저장
params to update[1]에 features.5.weight 저장
params to update[1]에 features.5.bias 저장
params to update[1]에 features.7.weight 저장
params to update[1]에 features.7.bias 저장
params to update[1]에 features.10.weight 저장
params to update[1]에 features.10.bias 저장
params to update[1]에 features.12.weight 저장
params to update[1]에 features.12.bias 저장
params to update[1]에 features.14.weight 저장
params to update[1]에 features.14.bias 저장
params to update[1]에 features.17.weight 저장
params to update[1]에 features.17.bias 저장
params to update[1]에 features.19.weight 저장
params to update[1]에 features.19.bias 저장
params to update[1]에 features.21.weight 저장
params to update[1]에 features.21.bias 저장
params to update[1]에 features.24.weight 저장
params to update[1]에 features.24.bias 저장
params to update[1]에 features.26.weight 저장
params to update[1]에 features.26.bias 저장


In [6]:
lr_list=[1e-4,5e-4,1e-3]

optimizer=optim.SGD([
    {'params':params_to_update[idx],'lr':lr} for idx,lr in enumerate(lr_list)
],momentum=0.9)

### training
https://sincerechloe.tistory.com/44

In [ ]:
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
    
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    net.to(device)

    torch.backends.cudnn.benchmark=True

    for epoch in range(num_epochs):
        print(f"Epoch{epoch+1}/{num_epochs}")
        for phase in ["train","val"]:
            if phase=="train":
                net.train()
            else: net.eval()

            epoch_loss=0.0 #loss 합
            epoch_correct=0 #정답 수

            if epoch==0 and phase=="train":
                continue
            
            for inputs,labels in tqdm(dataloaders_dict[phase]):

                inputs=inputs.to(device)
                labels=labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=="train"):
                    outputs=net(inputs)
                    loss=criterion(outputs,labels)
                    _,preds=torch.max(outputs,1)
                    if phase=="train":
                        loss.backward()
                        optimizer.step()
                    
                    epoch_loss+=loss.items()*inputs.size(0)
                    epoch_correct+=torch.sum(preds==labels.data)
            epoch_loss/=len(dataloaders_dict[phase].dataset)
            epoch_accuracy =epoch_correct.double()/len(dataloaders_dict[phase].dataset)

            print(f"{phase} Loss:{epoch_loss}, Accuracy:{epoch_accuracy}")            

In [8]:
num_epochs=2
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs)

cpu
Epoch1/2


  0%|          | 0/5 [00:00<?, ?it/s]

### save and load trained network

In [7]:
save_path='./weights_fine_tuning.pth'
torch.save(net.state_dict(),save_path)

NameError: name 'net' is not defined

In [ ]:
load_path='./weight_fine_tuing.pth'
load_weights=torch.load(load_path)
net.load_state_dict(load_weights)

load_weights=torch.load(load_path,map_location={'cuda:0':'cpu'})
net.load_state_dict(load_weights)